In [18]:
"""
🎯 LangGraph 파이프라인: 축제명 → 축제유형 탐지 → 동일 유형 축제 리스트
- 날짜는 무시합니다 (요청사항 반영).
- LangGraph .invoke() 결과(dict)를 FestState로 재변환하여 AttributeError 해결.

사용법(노트북 셀):
1) 이 코드를 한 셀에 붙여넣고 실행
2) 마지막 부분 예시:
    USER_FESTIVAL_NAME = "담양산타축제"
    result_state = run_pipeline(USER_FESTIVAL_NAME)
    display(result_state.same_type_df.head(50))

필요 패키지(한번만):
%pip install langgraph pydantic pandas
"""

import pandas as pd
import re
from typing import Optional, Tuple, List, Any
from difflib import get_close_matches
from pathlib import Path

from pydantic import BaseModel, Field
# Pydantic v2에서 DataFrame 같은 임의 타입 허용 설정
try:
    from pydantic import ConfigDict  # v2
except ImportError:
    ConfigDict = None

from langgraph.graph import StateGraph, START, END


# ----------------------------------------------------
# CSV 후보 경로 (상위에서부터 존재하는 첫 파일 사용)
# ----------------------------------------------------
CANDIDATE_PATHS: List[str] = [
    "./csv/축제_핵심필드_날짜완비_no지역.csv",
    "./csv/축제_핵심필드_날짜완비.csv",
    "./csv/축제_핵심필드.csv",
    "/mnt/data/csv/축제_핵심필드_날짜완비_no지역.csv",
    "/mnt/data/csv/축제_핵심필드_날짜완비.csv",
    "/mnt/data/csv/축제_핵심필드.csv",
    "/mnt/data/축제_핵심필드_날짜완비_no지역.csv",
    "/mnt/data/축제_핵심필드_날짜완비.csv",
    "/mnt/data/축제_핵심필드.csv",
]

ALLOWED_TYPES = ["문화예술", "지역특산물", "주민화합", "자연생태", "전통역사"]


# ---------------------------
# 유틸 함수
# ---------------------------
def resolve_csv_path(csv_path_override: Optional[str] = None) -> str:
    """지정 경로가 있으면 우선 사용, 없으면 후보 목록에서 존재하는 첫 파일 사용."""
    if csv_path_override and Path(csv_path_override).exists():
        return csv_path_override
    for p in CANDIDATE_PATHS:
        if Path(p).exists():
            return p
    raise FileNotFoundError("CSV 파일을 찾지 못했습니다. CANDIDATE_PATHS를 확인하세요.")

def normalize_text(s: str) -> str:
    if s is None:
        return ""
    s = str(s).strip()
    s = s.replace("\xa0", " ")
    return re.sub(r"\s+", "", s)

def ensure_columns(df: pd.DataFrame) -> pd.DataFrame:
    needed = ["연번", "광역자치단체명", "기초자치단체명", "축제명", "축제 유형"]
    for c in needed:
        if c not in df.columns:
            raise ValueError(f"필수 컬럼 누락: {c} (현재 보유: {list(df.columns)[:12]} ...)")
    return df

def detect_type_from_name(df: pd.DataFrame, name: str) -> Tuple[str, int, str]:
    """
    축제명을 기준으로 '축제 유형'을 찾는다.
    반환: (탐지된유형, 매칭된행수, 최종매칭축제명)
    매칭 순서: 정확일치 → 부분일치(공백무시) → 유사도(0.6)
    """
    # 1) 정확 일치
    exact = df[df["축제명"].astype(str).str.strip() == name.strip()]
    if not exact.empty:
        t = str(exact.iloc[0]["축제 유형"])
        return t, len(exact), str(exact.iloc[0]["축제명"])

    # 2) 부분 포함 (공백 제거 후)
    nname = normalize_text(name)
    cand = df[df["축제명"].astype(str).apply(lambda x: normalize_text(x) in nname or nname in normalize_text(x))]
    if not cand.empty:
        t = str(cand.iloc[0]["축제 유형"])
        return t, len(cand), str(cand.iloc[0]["축제명"])

    # 3) 유사도(상위 1개)
    all_names = df["축제명"].astype(str).tolist()
    hits = get_close_matches(name, all_names, n=1, cutoff=0.6)
    if hits:
        row = df[df["축제명"] == hits[0]].iloc[0]
        return str(row["축제 유형"]), 1, str(row["축제명"])

    return "", 0, ""


# ---------------------------
# LangGraph 상태/노드
# ---------------------------
class FestState(BaseModel):
    # ✅ Pydantic 설정 (DataFrame 같은 임의 타입 허용)
    if ConfigDict is not None:
        model_config = ConfigDict(arbitrary_types_allowed=True)  # Pydantic v2
    else:
        class Config:  # Pydantic v1 fallback
            arbitrary_types_allowed = True

    # 입력
    query_name: str = Field("", description="사용자가 입력한 축제명")
    csv_path_override: Optional[str] = None

    # 내부 (DataFrame은 임의 타입이므로 위 옵션 필요)
    df: Optional[pd.DataFrame] = None
    detected_type: str = ""
    detected_row_count: int = 0
    matched_name: str = ""

    # 출력
    same_type_df: Optional[pd.DataFrame] = None


def node_load_data(state: FestState) -> FestState:
    path = resolve_csv_path(state.csv_path_override)
    df = pd.read_csv(path, dtype=str).fillna("")
    state.df = ensure_columns(df)
    return state


def node_detect_type(state: FestState) -> FestState:
    assert state.df is not None, "데이터가 로드되지 않았습니다."
    dtype, cnt, mname = detect_type_from_name(state.df, state.query_name)
    state.detected_type = dtype
    state.detected_row_count = cnt
    state.matched_name = mname
    return state


def node_filter_same_type(state: FestState) -> FestState:
    assert state.df is not None, "데이터가 로드되지 않았습니다."
    if not state.detected_type:
        state.same_type_df = pd.DataFrame()
        return state

    same = state.df[state.df["축제 유형"].astype(str).str.strip() == state.detected_type.strip()].copy()

    # 보기 좋게 정렬
    same = same.sort_values(by=["광역자치단체명", "기초자치단체명", "축제명"], ascending=[True, True, True])

    # 출력 컬럼(요구사항: 날짜 무시)
    state.same_type_df = same[["연번", "광역자치단체명", "기초자치단체명", "축제명", "축제 유형"]]
    return state


# ---------------------------
# 그래프 구성 & 실행 헬퍼
# ---------------------------
def build_app():
    graph = StateGraph(FestState)
    graph.add_node("load_data", node_load_data)
    graph.add_node("detect_type", node_detect_type)
    graph.add_node("filter_same_type", node_filter_same_type)

    graph.add_edge(START, "load_data")
    graph.add_edge("load_data", "detect_type")
    graph.add_edge("detect_type", "filter_same_type")
    graph.add_edge("filter_same_type", END)

    return graph.compile()


APP = build_app()


def _to_state(obj) -> FestState:
    """LangGraph .invoke() 가 dict를 반환하는 경우 FestState로 변환."""
    if isinstance(obj, FestState):
        return obj
    try:
        # Pydantic v2
        return FestState.model_validate(obj)  # type: ignore[attr-defined]
    except Exception:
        try:
            # Pydantic v1
            return FestState.parse_obj(obj)  # type: ignore[attr-defined]
        except Exception:
            return FestState(**obj)  # 최후 수단


def run_pipeline(festival_name: str, csv_path_override: Optional[str] = None) -> FestState:
    """
    간편 실행:
      state = run_pipeline("담양산타축제")
      display(state.same_type_df.head(50))
    """
    init = FestState(query_name=festival_name, csv_path_override=csv_path_override)
    # LangGraph는 결과를 dict로 반환할 수 있으므로 변환
    raw = APP.invoke(init)
    final_state = _to_state(raw)

    # 콘솔 요약
    print("입력 축제명:", festival_name)
    print("매칭된 축제명(참고):", final_state.matched_name or "(없음)")
    print("탐지된 축제유형:", final_state.detected_type or "(탐지 실패)")
    print("입력명과 매칭된 행 수:", final_state.detected_row_count)
    print("동일 유형 축제 수:", (0 if final_state.same_type_df is None else len(final_state.same_type_df)))

    return final_state


# ---------------------------
# 스크립트로 직접 실행 시 (노트북에서도 동작)
# ---------------------------
if __name__ == "__main__":
    USER_FESTIVAL_NAME = "담양산타축제"
    state = run_pipeline(USER_FESTIVAL_NAME)
    # 결과 저장을 원하면 주석 해제
    # if state.same_type_df is not None and not state.same_type_df.empty:
    #     out = Path("./동일유형_전체목록.csv")
    #     state.same_type_df.to_csv(out, index=False, encoding="utf-8-sig")
    #     print("저장:", out.resolve())


입력 축제명: 담양산타축제
매칭된 축제명(참고): 제7회 담양산타축제
탐지된 축제유형: 문화예술
입력명과 매칭된 행 수: 2
동일 유형 축제 수: 294
